<a href="https://colab.research.google.com/github/esheshka/sber_SSAD/blob/main/train_model_sber_SSAD_v_last.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Скачивание файлов и подключение библиотек**

In [1]:
# @title **Скачивание файлов с Kaggle**
import urllib.request

!pip -q install --upgrade --force-reinstall --no-deps kaggle > log
link = urllib.request.urlopen("https://drive.usercontent.google.com/u/0/uc?id=1VEb54ANrxByHONXkngUpSjWMF0vKw2ba&export=download").read()
file = open("kaggle.json", "wb")
file.write(link)
file.close()

!mkdir -p ~/.kaggle
!mv /content/kaggle.json ~/.kaggle/kaggle.json > log
!chmod 600 /content/kaggle.json
!kaggle config set -n competition -v 26jun23
!kaggle competitions download >> log
!unzip -o *.zip >> log

chmod: cannot access '/content/kaggle.json': No such file or directory
- competition is now set to: 26jun23
100% 35.7M/35.7M [00:00<00:00, 44.7MB/s]


In [2]:
# @title **Подключение библиотек (предустановленных)**

%%time
%%capture
%reset -f

!pip install -U sentence-transformers tfds-nightly tensorflow tensorflow_addons plotly > log

from IPython.core.interactiveshell import InteractiveShell as IS
IS.ast_node_interactivity = "all"  # allows multiple prints from a cell

import numpy as np
import pandas as pd
pd.set_option('display.max_colwidth', 100, 'display.max_columns', 20, 'display.width', 1000, 'display.max_rows', 4)
os.environ['TF_DETERMINISTIC_OPS'] = '1'; os.environ['TF_CUDNN_DETERMINISTIC'] = '1'   # helps to seed TF's RNG on GPUs
np.set_printoptions(linewidth=10000, precision=2, edgeitems=20, suppress=True)
pd.set_option('display.max_colwidth', 100, 'display.max_columns', 10, 'display.width', 1000, 'display.max_rows', 8)

import time, matplotlib.pyplot as plt, os, plotly.express as px
import re

import xgboost as xgb, lightgbm
import tensorflow as tf
from sklearn.linear_model import LogisticRegression as LR
from sklearn.neural_network import MLPClassifier

from sklearn.preprocessing import MinMaxScaler, PolynomialFeatures
from sklearn.model_selection import train_test_split, GroupKFold
from sklearn.metrics import roc_auc_score
from sklearn.decomposition import PCA
from sentence_transformers import SentenceTransformer as SBERT

import tensorflow.keras as keras
from keras.layers import Flatten, Dense

CPU times: user 13.5 s, sys: 1.89 s, total: 15.4 s
Wall time: 1min 2s


In [3]:
# @title **Таймера**

class Timer():
  def __init__(self, lim:'RunTimeLimit'=60*2): self.t0, self.lim, _ = time.time(), lim, print(f'⏳ started. You have {lim} sec. Good luck!')
  def ShowTime(self):
    msg = f'Runtime is {time.time()-self.t0:.0f} sec'
    print(f'\033[91m\033[1m' + msg + f' > {self.lim} sec limit!!!\033[0m' if (time.time()-self.t0-1) > self.lim else msg)

# **Чтение данных**



In [37]:
mcc = pd.read_csv('_mcc.csv', index_col='mcc')

ttc = pd.read_csv('_ttc.csv', index_col='ttc')

gender = pd.read_csv('gender.csv', index_col='cid')
trainY, testY = gender.query('gender==gender'), gender.query('gender!=gender')

dfTrx = pd.read_csv('trx.csv').sort_values(['cid','dt','tm'])


df_ = gender.drop('gender', axis=1); df_.T  # a list of ordered client IDs (as index)

cid,0,1,2,3,4,...,8395,8396,8397,8398,8399


In [38]:
# @title **Включение таймера**

tmr = Timer()

⏳ started. You have 120 sec. Good luck!


# **Начало кода (таймер пошел)**

In [128]:
# @title **Подключение библиотек (дополнительных)**

from sklearn.ensemble import RandomForestClassifier as RFC
from sklearn.ensemble import GradientBoostingClassifier as GBC

from sklearn.neural_network import MLPClassifier as MLPC

from sklearn.model_selection import GridSearchCV as GSCV

!pip install lightgbm
!pip install catboost

from lightgbm import LGBMClassifier as LGBMC
from catboost import CatBoostClassifier as CBC

import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing

# **Поиск частовстречающихся значений для разных полов**

In [40]:
df_test = dfTrx.join(gender, on='cid', how='inner')
df_test = df_test[~df_test.gender.isna()]

In [41]:
#@title **MCC**

mcc_list = list(set(df_test.mcc))
df_mcc = pd.DataFrame({'mcc': mcc_list})

# Пол со значением 0
zero = df_test[df_test.gender == 0][['cid', 'mcc']].drop_duplicates().groupby('mcc').agg('size').fillna(0)
zero = df_mcc.join(pd.DataFrame({'zero': zero}), on='mcc', how='outer').fillna(0)

# Пол со значением 1
one = df_test[df_test.gender == 1][['cid', 'mcc']].drop_duplicates().groupby('mcc').agg('size').fillna(0)
one = df_mcc.join(pd.DataFrame({'one': one}), on='mcc', how='outer').fillna(0)

df_mcc = df_mcc.join(pd.DataFrame({'count': (zero.zero + one.one)}), on='mcc', how='left').join(pd.DataFrame({'%': zero.zero / (zero.zero + one.one) * 100}), on='mcc', how='left')

# Сет наиболее важных значений
interesting_mcc = set(df_mcc[(df_mcc['count'] > 100) & ((df_mcc['%'] > 70) | (df_mcc['%'] < 30))].mcc)
interesting_mcc

{62, 63, 70, 71, 82, 106, 149, 150, 161}

In [42]:
#@title **TTC**

ttc_list = list(set(df_test.ttc))
df_ttc = pd.DataFrame({'ttc': ttc_list})

# Пол со значением 0
zero = df_test[df_test.gender == 0][['cid', 'ttc']].drop_duplicates().groupby('ttc').agg('size').fillna(0)
zero = df_ttc.join(pd.DataFrame({'zero': zero}), on='ttc', how='outer').fillna(0)

# Пол со значением 1
one = df_test[df_test.gender == 1][['cid', 'ttc']].drop_duplicates().groupby('ttc').agg('size').fillna(0)
one = df_ttc.join(pd.DataFrame({'one': one}), on='ttc', how='outer').fillna(0)

df_ttc = df_ttc.join(pd.DataFrame({'count': (zero.zero + one.one)}), on='ttc', how='left').join(pd.DataFrame({'%': zero.zero / (zero.zero + one.one) * 100}), on='ttc', how='left')

# Сет наиболее важных значений
interesting_ttc = set(df_ttc[(df_ttc['count'] > 100) & ((df_ttc['%'] > 70) | (df_ttc['%'] < 30))].ttc)
interesting_ttc

set()

In [43]:
#@title **TID**

tid_list = list(set(df_test.tid))
df_tid = pd.DataFrame({'tid': tid_list})

# Пол со значением 0
zero = df_test[df_test.gender == 0][['cid', 'tid']].drop_duplicates().groupby('tid').agg('size').fillna(0)
zero = df_tid.join(pd.DataFrame({'gender': zero}), on='tid', how='outer').fillna(0).astype({'gender': int})

# Пол со значением 1
one = df_test[df_test.gender == 1][['cid', 'tid']].drop_duplicates().groupby('tid').agg('size').fillna(0)
one = df_tid.join(pd.DataFrame({'gender': one}), on='tid', how='outer').fillna(0).astype({'gender': int})

df_tid = df_tid
df_tid = df_tid.join(zero[['gender']] + one[['gender']]).rename(columns={'gender': 'count'}).join(100 * zero[['gender']] / (zero[['gender']] + one[['gender']])).rename(columns={'gender': '%'})

# Сет наиболее важных значений
interesting_tid = set(df_tid[(df_tid['count'] > 100) & ((df_tid['%'] > 70) | (df_tid['%'] < 30))].tid)
interesting_tid

{'10244893', '20001004', '20001033', '20828301', '940047'}

# **Поиск частовстречающихся значений**

In [321]:
#@title **Параметры и коэффиценты**

large_count = 2000
large_sum = -1000000000

k = 1
k1 = 0.8
k2 = 0.85
k3 = 8

In [322]:
#@title **Определение наиболее используемых mcc**

%%time
mcc_by_cid = dfTrx[['cid', 'mcc']].drop_duplicates().astype({'mcc': str}).groupby('cid').agg({'mcc': ' '.join}).sort_values('cid')

large_mcc_count = dfTrx[['cid', 'mcc']].drop_duplicates().groupby('mcc').agg({'cid': 'size'}).sort_values(by = 'cid')
large_mcc_sum = dfTrx[['amt', 'mcc']].groupby('mcc').agg('sum').sort_values(by = 'amt')

most_popular_mcc = set(large_mcc_count[large_mcc_count.cid > large_count * k1].index.to_list()) | set(large_mcc_sum[large_mcc_sum.amt < large_sum * k1].index.to_list()) | set([123, 125, 130])
len(most_popular_mcc)

CPU times: user 665 ms, sys: 123 ms, total: 789 ms
Wall time: 787 ms


36

In [323]:
#@title **Определение наиболее используемых ttc**

%%time
ttc_by_cid = dfTrx[['cid', 'ttc']].drop_duplicates().astype({'ttc': str}).groupby('cid').agg({'ttc': ' '.join}).sort_values('cid')

large_ttc_count = dfTrx[['cid', 'ttc']].drop_duplicates().groupby('ttc').agg({'cid': 'size'}).sort_values(by = 'cid')
large_ttc_sum = dfTrx[['amt', 'ttc']].groupby('ttc').agg('sum').sort_values(by = 'amt')

most_popular_ttc = set(large_ttc_count[large_ttc_count.cid > large_count * k2].index.to_list()) | set(large_ttc_sum[(large_ttc_sum.amt < large_sum * k2) | (large_ttc_sum.amt > 1000000000)].index.to_list())
len(most_popular_ttc)

CPU times: user 597 ms, sys: 147 ms, total: 744 ms
Wall time: 742 ms


25

In [324]:
#@title **Определение наиболее используемых tid**

%%time
tid_by_cid = dfTrx[['cid', 'tid']].drop_duplicates().astype({'tid': str}).groupby('cid').agg({'tid': ' '.join}).sort_values('cid')

large_tid_count = dfTrx[['cid', 'tid']].drop_duplicates().groupby('tid').agg({'cid': 'size'}).sort_values(by = 'cid')
large_tid_sum = dfTrx[['amt', 'tid']].groupby('tid').agg('sum').sort_values(by = 'amt')

most_popular_tid = set(large_tid_count[large_tid_count.cid > large_count * k3].index.to_list()) | set(large_tid_sum[(large_tid_sum.amt < large_sum * k3) | (large_tid_sum.amt > 1000000000)].index.to_list())
len(most_popular_tid)

CPU times: user 5.65 s, sys: 225 ms, total: 5.87 s
Wall time: 6 s


0

# **Создания единого df**

In [340]:
#@title **Сумма**

def s(to_df):
    return to_df / to_df.sum(axis=0)

In [341]:
#@title **Параметры и коэффиценты**

# stat = ['count', 'sum', 'median']
stat = ['count', 'sum']
# stat = ['count']

In [342]:
%%time

df = df_.copy(deep=True)

CPU times: user 198 µs, sys: 11 µs, total: 209 µs
Wall time: 194 µs


In [343]:
#@title **MCC**

%%time
to_df = df_.copy(deep=True)

for mcc_index in (most_popular_mcc - interesting_mcc):
    stat_amt_by_cid_and_mcc = dfTrx[dfTrx.mcc == mcc_index][['cid', 'amt']].groupby('cid').agg(stat).sort_values(by = 'cid').add_prefix('').droplevel(0, axis='columns')
    stat_amt_by_cid_and_mcc.columns += '_' + str(mcc_index) + '_mcc'
    to_df = to_df.join(stat_amt_by_cid_and_mcc, how='left')

stat_by_other_mcc = dfTrx[~dfTrx.mcc.isin(most_popular_mcc or interesting_mcc)][['cid', 'amt']].groupby('cid').agg(stat).sort_values(by = 'cid').add_prefix('').droplevel(0, axis='columns')
stat_by_other_mcc.columns += '_mcc'
to_df = to_df.join(stat_by_other_mcc, how='left')

to_df = s(to_df) * 75
df = df.join(to_df, on = 'cid', how='left')
to_df = df_.copy(deep=True)

for mcc_index in interesting_mcc:
    stat_amt_by_cid_and_mcc = dfTrx[dfTrx.mcc == mcc_index][['cid', 'amt']].groupby('cid').agg(stat).sort_values(by = 'cid').add_prefix('').droplevel(0, axis='columns')
    stat_amt_by_cid_and_mcc.columns += '_' + str(mcc_index) + '_mcc'
    to_df = to_df.join(stat_amt_by_cid_and_mcc, how='left')

to_df = s(to_df) * 200
df = df.join(to_df, on = 'cid', how='left')

CPU times: user 1.96 s, sys: 36.1 ms, total: 1.99 s
Wall time: 3.38 s


In [344]:
#@title **TTC**
to_df = df_.copy(deep=True)

for ttc_index in (most_popular_ttc - interesting_ttc):
    stat_amt_by_cid_and_ttc = dfTrx[dfTrx.ttc == ttc_index][['cid', 'amt']].groupby('cid').agg(stat).sort_values(by = 'cid').add_prefix('').droplevel(0, axis='columns')
    stat_amt_by_cid_and_ttc.columns += '_' + str(ttc_index) + '_ttc'
    to_df = to_df.join(stat_amt_by_cid_and_ttc, how='left')

stat_by_other_ttc = dfTrx[~dfTrx.ttc.isin(most_popular_ttc or interesting_ttc)][['cid', 'amt']].groupby('cid').agg(stat).sort_values(by = 'cid').add_prefix('').droplevel(0, axis='columns')
stat_by_other_ttc.columns += '_ttc'
to_df = to_df.join(stat_by_other_ttc, how='left')

to_df = s(to_df) * 1/2
df = df.join(to_df, on = 'cid', how='left')
to_df = df_.copy(deep=True)

for ttc_index in (interesting_ttc):
    stat_amt_by_cid_and_ttc = dfTrx[dfTrx.ttc == ttc_index][['cid', 'amt']].groupby('cid').agg(stat).sort_values(by = 'cid').add_prefix('').droplevel(0, axis='columns')
    stat_amt_by_cid_and_ttc.columns += '_' + str(ttc_index) + '_ttc'
    to_df = to_df.join(stat_amt_by_cid_and_ttc, how='left')

to_df = s(to_df) * 50
df = df.join(to_df, on = 'cid', how='left')

In [345]:
#@title **AMT**

to_df = df_.copy(deep=True)

to_df = dfTrx[['cid', 'amt']].groupby('cid').agg(stat).add_prefix('').droplevel(0, axis='columns')
to_df = df_.join(to_df, how='left').fillna(0)

to_df = s(to_df) * 2/3
df = df.join(to_df, on = 'cid', how='left')

In [346]:
#@title **Time**

%%time
to_df = df_.copy(deep=True)

for hour in range(0, 1):
    stat_amt_by_cid_and_hour = dfTrx[dfTrx.tm == hour][['cid', 'amt']].groupby('cid').agg(stat).sort_values(by = 'cid').add_prefix('').droplevel(0, axis='columns')
    stat_amt_by_cid_and_hour.columns += '_' + str(hour) + '_hour'
    to_df = to_df.join(stat_amt_by_cid_and_hour, how='left')

to_df = s(to_df) * 2
df = df.join(to_df, on = 'cid', how='left')
to_df = df_.copy(deep=True)

for hour in range(1, 9):
    stat_amt_by_cid_and_hour = dfTrx[dfTrx.tm == hour][['cid', 'amt']].groupby('cid').agg(stat).sort_values(by = 'cid').add_prefix('').droplevel(0, axis='columns')
    stat_amt_by_cid_and_hour.columns += '_' + str(hour) + '_hour'
    to_df = to_df.join(stat_amt_by_cid_and_hour, how='left')

to_df = s(to_df)
df = df.join(to_df, on = 'cid', how='left')

CPU times: user 4.23 s, sys: 51.8 ms, total: 4.28 s
Wall time: 4.26 s


In [347]:
#@title **Date (Анализ дней недели)**

%%time
to_df = df_.copy(deep=True)
to_df = s(to_df)
df = df.join(to_df, on = 'cid', how='left')

to_df = df_.copy(deep=True)

for weekday in [1, 2]:
    stat_amt_by_cid_and_weekday = dfTrx[dfTrx.dt % 7 == weekday][['cid', 'amt']].groupby('cid').agg(stat).sort_values(by = 'cid').add_prefix('').droplevel(0, axis='columns')
    stat_amt_by_cid_and_weekday.columns += '_' + str(weekday) + '_wd'
    to_df = to_df.join(stat_amt_by_cid_and_weekday, on = 'cid', how='left')

to_df = s(to_df)
df = df.join(to_df, on = 'cid', how='left')

CPU times: user 305 ms, sys: 16 ms, total: 321 ms
Wall time: 318 ms


In [348]:
#@title **TID**
to_df = df_.copy(deep=True)

for tid_index in (most_popular_tid - interesting_tid):
    stat_amt_by_cid_and_tid = dfTrx[dfTrx.tid == tid_index][['cid', 'amt']].groupby('cid').agg(stat).sort_values(by = 'cid').add_prefix('').droplevel(0, axis='columns')
    stat_amt_by_cid_and_tid.columns += '_' + str(tid_index) + '_tid'
    to_df = to_df.join(stat_amt_by_cid_and_tid, how='left')

stat_by_other_tid = dfTrx[~dfTrx.tid.isin(most_popular_tid or interesting_tid)][['cid', 'amt']].groupby('cid').agg(stat).sort_values(by = 'cid').add_prefix('').droplevel(0, axis='columns')
stat_by_other_tid.columns += '_tid'
to_df = to_df.join(stat_by_other_tid, how='left')

to_df = s(to_df)
df = df.join(to_df, on = 'cid', how='left')

to_df = df_.copy(deep=True)

for tid_index in (interesting_tid):
    stat_amt_by_cid_and_tid = dfTrx[dfTrx.tid == tid_index][['cid', 'amt']].groupby('cid').agg(stat).sort_values(by = 'cid').add_prefix('').droplevel(0, axis='columns')
    stat_amt_by_cid_and_tid.columns += '_' + str(tid_index) + '_tid'
    to_df = to_df.join(stat_amt_by_cid_and_tid, how='left')

to_df = s(to_df) * 50
df = df.join(to_df, on = 'cid', how='left')

In [349]:
df = df.fillna(0).astype('float')
df.columns = [str(c) for c in df.columns]

# **Подготовка к обучение**

In [351]:
#@title **Нормализация**

scaler = preprocessing.StandardScaler().fit(df)

df = pd.DataFrame(scaler.transform(df), columns=df.columns)

In [352]:
#@title **Проверка корреляции**

# num_cols = df._get_numeric_data().columns

# correlation_matrix = np.corrcoef(df[num_cols], rowvar=False)

# plt.figure(figsize=(12, 10))
# sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', xticklabels=df[num_cols].columns, yticklabels=df[num_cols].columns)

# plt.title('Correlation Matrix')

# plt.show()

In [353]:
#@title **Z-SCORE проверка на выбросы**

def Zscore_outlier(df):
    out=[]
    m = np.mean(df)
    sd = np.std(df)
    for i in range(len(df)):
        z = (df.iloc[[i]].values[0] - m) / sd
        if np.abs(z) > 3:
            out.append(df.iloc[[i]].index[0])
    return out


# outliers = set()
# iteration = 0
# for col in df.columns:
#     print(iteration, "/", len(df.columns))
#     iteration += 1
#     out = Zscore_outlier(df[col])
#     outliers.update(out)

# outliers = list(outliers)
# print(len(outliers), "rows were deleted")
# df.drop(index=outliers, inplace=True)

In [354]:
%%time
df.columns = [str(c) for c in df.columns]

trainX, testX = df.join(trainY.drop('gender', axis=1), how='inner'), df.join(testY.drop('gender', axis=1), how='inner')
trainX0, testX0, trainY0, testY0 = train_test_split(trainX, trainY, test_size=0.28, random_state=0)


CPU times: user 22.2 ms, sys: 0 ns, total: 22.2 ms
Wall time: 22.3 ms


# **Обучение**

In [355]:
model_final = MLPC(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(7, 4), random_state=1)

model1 = MLPC(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(7, 4), random_state=1)
model2 = GBC(loss='log_loss', learning_rate=0.1, n_estimators=150, subsample=1.0, criterion='friedman_mse', max_depth=2)
model3 = CBC(iterations=400)
model4 = LGBMC()

pY1 = model1.fit(trainX0, trainY0.gender).predict_proba(testX0)[:, 1]
pY2 = model2.fit(trainX0, trainY0.gender).predict_proba(testX0)[:, 1]
pY3 = model3.fit(trainX0, trainY0.gender).predict_proba(testX0)[:, 1]
pY4 = model4.fit(trainX0, trainY0.gender).predict_proba(testX0)[:, 1]

/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:545: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


Learning rate set to 0.045827
0:	learn: 0.6787336	total: 82.5ms	remaining: 32.9s
1:	learn: 0.6660776	total: 140ms	remaining: 27.9s
2:	learn: 0.6547993	total: 209ms	remaining: 27.7s
3:	learn: 0.6450134	total: 269ms	remaining: 26.7s
4:	learn: 0.6353197	total: 324ms	remaining: 25.6s
5:	learn: 0.6262667	total: 386ms	remaining: 25.4s
6:	learn: 0.6184837	total: 453ms	remaining: 25.4s
7:	learn: 0.6110271	total: 519ms	remaining: 25.4s
8:	learn: 0.6026752	total: 581ms	remaining: 25.2s
9:	learn: 0.5958769	total: 649ms	remaining: 25.3s
10:	learn: 0.5902521	total: 710ms	remaining: 25.1s
11:	learn: 0.5834575	total: 768ms	remaining: 24.8s
12:	learn: 0.5778690	total: 833ms	remaining: 24.8s
13:	learn: 0.5730356	total: 902ms	remaining: 24.9s
14:	learn: 0.5677820	total: 949ms	remaining: 24.4s
15:	learn: 0.5631334	total: 1.03s	remaining: 24.8s
16:	learn: 0.5584366	total: 1.1s	remaining: 24.9s
17:	learn: 0.5542918	total: 1.17s	remaining: 24.9s
18:	learn: 0.5513034	total: 1.25s	remaining: 25s
19:	learn: 0.

In [356]:
pY = (pY1 + 2*pY2 + 4*pY3 + 2*pY4) / 9

print('Local AUC ROC', roc_auc_score(testY0.gender, pY))
print(pY.sum() / pY.shape[0])

Local AUC ROC 0.8694118241280906
0.4341806133673879


In [357]:
X_t = np.array([pY1, pY2, pY3, pY4]).T

train_X_t, test_X_t, train_Y_t, test_Y_t = train_test_split(X_t, testY0, test_size=0.3, random_state=0)


p_t = model1.fit(train_X_t, train_Y_t.gender).predict_proba(test_X_t)[:, 1]

print('Local AUC ROC', roc_auc_score(test_Y_t.gender, p_t))

Local AUC ROC 0.8640027196623086


In [358]:
tmr.ShowTime()

Runtime is 6285 sec > 120 sec limit!!!
